In [114]:
import pandas as pd
import re
import numpy

df = pd.read_csv('results-survey159448.csv')

# only get relevant columns
df_processed = df.filter(regex='Please_read_each_of_the_following_items_carefully,_thinking_about_how_it_relates_to_dealing_with_usability_issues_in_open_source application_software,_and_then_indicate_how_true_it_is_for_you._|Response_ID')

# remove the common part
df_processed.columns = df_processed.columns.str.replace('Please_read_each_of_the_following_items_carefully,_thinking_about_how_it_relates_to_dealing_with_usability_issues_in_open_source application_software,_and_then_indicate_how_true_it_is_for_you._','')


# rename column, so they are put into proper order to calculates stats
df_processed = df_processed.rename(index=str, 
           columns={'[I_feel_like_I_can_make_a_lot_of_inputs_to_deciding_how_the_job_gets_done.]': 'Question1',
                    '[I_really_like_the_people_I_interact_with.]': 'Question2',
                    '[Often,_I_do_not_feel_very_competent.]': 'Question3',
                    '[I_feel_pressured_when_dealing_with_these_issues.]': 'Question5',
                    '[People_tell_me_that_I_am_good_at_what_I_do.]': 'Question4',
                    '[I_get_along_with_people_I_come_into_contact_with.]': 'Question6',
                    '[I_pretty_much_keep_to_myself_when_dealing_with_these_issues.]': 'Question7',
                    '[I_generally_feel_free_to_express_my_ideas_and_opinions.]': 'Question8',
                    '[I_consider_the_people_I_regularly_interact_with_to_be_my_friends.]': 'Question9',
                    '[I_have_been_able_to_learn_interesting_new_skills_recently.]': 'Question10',
                    '[I_frequently_have_to_do_what_I_am_told.]': 'Question11',
                    '[People_in_the_community_care_about_me.]': 'Question15',
                    '[Most_days_I_feel_a_sense_of_accomplishment_from_what_I_do.]': 'Question12',
                    '[People_I_interact_with_tend_to_take_my_feelings_into_consideration.]': 'Question13',
                    '[I_do_not_get_much_of_a_chance_to_show_how_capable_I_am.]': 'Question14',
                    '[There_are_not_many_people_that_I_am_close_to.]': 'Question16',
                    '[I_feel_like_I_can_pretty_much_be_myself_when_dealing_with_these_issues.]': 'Question17',
                    '[The_people_I_interact_with_do_not_seem_to_like_me_much.]': 'Question18',
                    '[I_often_do_not_feel_very_capable.]': 'Question19',
                    '[There_is_not_much_opportunity_for_me_to_decide_for_myself_how_to_do.]': 'Question20',
                    '[People_are_generally_pretty_friendly_towards_me.]': 'Question21'})

# with pd.option_context('display.max_rows', None, 'display.max_columns', 50):
#     print(df_processed)

df_num = df_processed.fillna(4)
df_num = df_num.replace('A1', 1)
df_num = df_num.replace('A2', 2)
df_num = df_num.replace('A3', 3)
df_num = df_num.replace('A4', 4)
df_num = df_num.replace('A5', 5)
df_num = df_num.replace('A6', 6)
df_num = df_num.replace('A7', 7)

# calculate Autonomy
# average of 1, 5(R), 8, 11(R), 13, 17, 20(R)
def calcu_autonomy(row):
    stat_list = []
    stat_list.append(row['Question1'])
    stat_list.append(8 - row['Question5'])
    stat_list.append(row['Question8'])
    stat_list.append(8 - row['Question11'])
    stat_list.append(row['Question13'])
    stat_list.append(row['Question17'])
    stat_list.append(8 - row['Question20'])
    return float(sum(stat_list))/len(stat_list)

df_processed['autonomy'] = df_num.apply(lambda row: calcu_autonomy(row), axis=1)

# # calculate Competence
# # average of 3(R), 4, 10, 12, 14(R), 19(R)
def calcu_competence(row):
    stat_list = []
    stat_list.append(8 - row['Question3'])
    stat_list.append(row['Question4'])
    stat_list.append(row['Question10'])
    stat_list.append(row['Question12'])
    stat_list.append(8 - row['Question14'])
    stat_list.append(8 - row['Question19'])
    return float(sum(stat_list))/len(stat_list)

df_processed['competence'] = df_num.apply(lambda row: calcu_competence(row), axis=1)

# # calculate Relatedness
# # average of 2, 6, 7(R), 9, 15, 16(R), 18(R), 21
def calcu_relatedness(row):
    stat_list = []
    stat_list.append(row['Question2'])
    stat_list.append(row['Question6'])
    stat_list.append(8 - row['Question7'])
    stat_list.append(row['Question9'])
    stat_list.append(row['Question15'])
    stat_list.append(8 - row['Question16'])
    stat_list.append(8 - row['Question18'])
    stat_list.append(row['Question21'])
    return float(sum(stat_list))/len(stat_list)

df_processed['relatedness'] = df_num.apply(lambda row: calcu_relatedness(row), axis=1)


with pd.option_context('display.max_rows', None, 'display.max_columns', 50):
    print(df_processed)

# # write to file
df_processed.to_csv("scoring.csv", sep=',', encoding='utf-8', index=False)



    Response_ID Question1 Question2 Question3 Question5 Question4 Question6  \
0             9        A4        A6        A2        A3        A6        A7   
1            13        A6        A5        A4        A5        A2        A4   
2            15        A7        A7        A4        A5        A5        A7   
3            17        A7        A6        A4        A2        A6        A7   
4            19        A7        A7        A2        A1        A7        A7   
5            20        A6        A7        A6        A1        A1        A6   
6            23        A4        A4        A4        A2       NaN        A4   
7            24        A3        A6        A6        A2        A5        A6   
8            33        A7        A7        A4        A4        A6        A6   
9            40        A5        A4        A5        A4        A3        A5   
10           42        A4        A4        A4        A4        A4        A4   
11           43        A7        A4        A4       